In [23]:
import pandas as pd
import numpy as np
import pickle
import speech_recognition as sr
import joblib
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score

In [24]:
# ✅ Step 1: Load and Preprocess Dataset
file_path = "pd_speech_features.csv"  # Ensure the file is in the same directory
df = pd.read_csv(file_path, skiprows=1)  # Skip the first row (incorrect headers)

# Separate features and target
X = df.drop(columns=['class'])  # Features (all except 'class')
y = df['class']  # Target (0 = Healthy, 1 = Parkinson's)

In [25]:
# ✅ Step 2: Split Data into Training and Testing Sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# ✅ Step 3: Train Model
model = RandomForestClassifier(n_estimators=100, random_state=42)
model.fit(X_train, y_train)

RandomForestClassifier(random_state=42)

In [26]:
# ✅ Step 4: Evaluate Model
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Model Accuracy: {accuracy:.2f}")

# ✅ Step 5: Save the Trained Model
model_filename = "parkinsons_model.pkl"
joblib.dump(model, model_filename)
print(f"Model saved as {model_filename}")

Model Accuracy: 0.88
Model saved as parkinsons_model.pkl


In [27]:
# ✅ Step 6: Prediction for Two New Patients with Symptom Detection
def detect_early_signs(features):
    """
    Detects early Parkinson’s signs based on feature values.
    These thresholds are hypothetical and should be adjusted based on feature significance.
    """
    tremor_threshold = 0.5  # Hypothetical threshold for voice tremors
    cough_threshold = 0.3   # Hypothetical threshold for cough-like disturbances
    speech_irregularity_threshold = 0.6  # Hypothetical threshold for speech irregularities
    
    signs = []
    if features[0] > tremor_threshold:  # Assuming first feature is tremor-related
        signs.append("Voice Tremors")
    if features[1] > cough_threshold:  # Assuming second feature is cough-related
        signs.append("Cough-like disturbances")
    if features[2] > speech_irregularity_threshold:  # Assuming third feature is speech irregularity
        signs.append("Speech Irregularities")
    
    return ", ".join(signs) if signs else "None"

In [28]:
# ✅ Step 6: Prediction for Two New Patients
new_patients = np.array([
    X.iloc[0].values,  # Assume first row has Parkinson’s
    X.iloc[1].values   # Assume second row is healthy
])

In [29]:
predictions = model.predict(new_patients)

c:\Users\balas\Anaconda3\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but RandomForestClassifier was fitted with feature names
  warnings.warn(


In [30]:
# Displaying formatted output with early sign detection
print("\n**Prediction for Two New Patients**")
for i, pred in enumerate(predictions):
    actual_label = "Parkinson’s" if y.iloc[i] == 1 else "Healthy"
    predicted_label = "Parkinson’s" if pred == 1 else "Healthy"
    signs_detected = detect_early_signs(new_patients[i])
    
    print(f"**Patient {i+1}** (Actual: {actual_label}) → **Predicted: {predicted_label}** | Signs Detected: {signs_detected}")


**Prediction for Two New Patients**
**Patient 1** (Actual: Parkinson’s) → **Predicted: Parkinson’s** | Signs Detected: Cough-like disturbances, Speech Irregularities
**Patient 2** (Actual: Parkinson’s) → **Predicted: Parkinson’s** | Signs Detected: Cough-like disturbances, Speech Irregularities


In [32]:
# ✅ Step 7: Real-time Speech Input for Parkinson’s Detection
def analyze_speech():
    recognizer = sr.Recognizer()
    with sr.Microphone() as source:
        print("\n🎤 Speak Now: (System will analyze your speech to detect Parkinson’s)")
        recognizer.adjust_for_ambient_noise(source)
        audio = recognizer.listen(source)
        
        try:
            text = recognizer.recognize_google(audio)
            print(f"🔊 Recognized Speech: {text}")
            
            # Simple rule-based logic: If speech is unclear, predict Parkinson's
            if "uhhh" in text or "cough" in text or len(text.split()) < 3:
                print("🩺 **Predicted: Parkinson’s** ❌")
            else:
                print("🩺 **Predicted: Healthy** ✅")
        except sr.UnknownValueError:
            print("⚠️ Speech could not be understood. Try again.")
        except sr.RequestError:
            print("⚠️ Could not request results. Check your internet connection.")

In [33]:
# Run real-time speech analysis
analyze_speech()


🎤 Speak Now: (System will analyze your speech to detect Parkinson’s)
🔊 Recognized Speech: hi
🩺 **Predicted: Parkinson’s** ❌
